# Variables, expressions, and operator precedence

## Introduction

In the previous chapter we finished by building a fake programming language, _Turtle_, with a very simple state which always contains exactly three pieces of information: `x`, `y`, and `pen`.

When solving bigger problems, it stands to reason that the state of _Turtle_ would fall dramatically short: how could we store information such as the year of birth of more than two people? Even if we accepted the misuses of names `x` and `y` as years of birth, `pen` can only be `on`/`off`, therefore is inadequate to represent a year.

In this chapter we will define a new simplistic beginner's programming language, which this time will be capable of more explicitly managing the state by also defining new names to store information in the state.

## Variables

When we define a new _name_, such as `year_of_birth`, which we subsequently use to bind and retrieve values in the state multiple times while running a program, we have implicitly defined what is almost universally called a _variable_. The term _variable_ comes from the fact that, by consistently using the same name, we are effectively defining a permanent meta-entity that will accompany the evaluation of the whole program. While the program is evaluated, the value bound to the name _changes_ (or _varies_), hence the name.

We now define a new, simple language, _Var_, which only allows us to perform variable manipulation. The syntax of the statements of _Var_ is:
- `V := N`, where `V` is a variable name and `N` is an arbitrary integer number (this is called _variable assignment_);
- `I;J`, where `I` and `J` are arbitrary statements (right associative: `A;B;C` means `A;(B;C)`);
- `done`.

The semantics of `Var` simply translates variable assignment to state binding, and follows the semicolons through:
- `eval(<V := N>, S)` $\rightarrow$ `<done>, S[V := N]`;
- `eval(<I; J>, S)` $\rightarrow$ `<I';J>, S'` given that `eval(<I>,S)` $\rightarrow$ `<I'>, S'` and `I` $\neq$ `done`;
- `eval(<done; I>, S)` $\rightarrow$ `<I>, S`;
- `eval(<done>, S)` $\rightarrow$ `<done>, S`.

Notice that variable assignment is just a way to _embed_ the binding operation, which operates on states, in the language itself. All instructions, no matter how advanced, are ways to embed the existing capabilities of state manipulation into the language: the language can not be more powerful than the underlying mechanisms upon which it is based.

Let us now consider an example program we could write in `Var`:

```day := 2; month := 3; year := 1985; done```

The first state is, as usual, the empty state: 

$$\{ \}$$

The evaluation will first run the `;` statement: `day := 2; ...` . This means that we begin by evaluating the left operand `day := 2` on an empty state. The semantic rules for assignment tell us that this leads us to a new pair program, state of:

```month := 3; year := 1985; done```

and

$\{ day := 2 \}$

Notice that variable assignment just allows us to write some statements that will directly, and without further translation, become bindings in the state. It is a transformation that reflects the structure of bindings in the program code itself.

## Expressions

The _Var_ language so far is not very useful by itself. We can store information in the state, but storage without the possibility of retrieval is not very useful. Fortunately, the state already supports reading of bindings, therefore we only need to embed an operation that is equivalent to state lookups in _Var_ and we are done.

In order to perform something useful, we will not just lookup variables directly. We will also perform some simple arithmetic operations while performing the lookup, therefore augmenting lookup with some basic transformations at the same time.

The assignment statement will therefore become:
- `V := E`, where `V` is a variable name and `E` is an _integer expression_;
- an integer expression can now be:
    - `N`, where `N` is an integer number;
    - `E`<sub>1</sub> ` + E`<sub>2</sub>, where both `E`<sub>1</sub> and `E`<sub>2</sub> are integer expressions;
    - `E`<sub>1</sub> ` - E`<sub>2</sub>, where both `E`<sub>1</sub> and `E`<sub>2</sub> are integer expressions;
    - `V`, where `V` is a variable name;

Examples of allowed integer expressions according to the definition above could therefore be:
- `3 + 4`
- `age + 1`
- `x + x`
- `new_score + old_score`

Of course, assigning an expression to a variable cannot be done just directly. Our state is built up from bindings of names to values, so our state is not powerful or expressive enough to store an expression. Moreover, storing an expression directly would not be so interesting: we would rather like to evaluate the expression, and then store the resulting value in the state.

Fortunately, in our language, all expressions can be mapped to integer values by means of the _expression evaluation_ function, which we will call `eval_expr`. Evaluation of an expression requires the ability to read from the state, since an expression might contain variable names that can only be evaluated by a state lookup. When the expression is done evaluating, the result is either a new expression to evaluate further, or the final value, but no new state: expressions do not produce a new state when evaluating. This means that `eval_expr(<E>,S)` $\rightarrow$ `<E'> | V`, where `E'` is a new expression, `V` is a value, and `|` means "either of".

The semantics of expressions can therefore be stated as:
- `eval_expr(<N>, S)` $\rightarrow$ `N` (`N` is a number);
- `eval_expr(<V>, S)` $\rightarrow$ `C`, where `V` is a variable name, and `S[V]` $\rightarrow$ `C`;
- `eval_expr(<L + R>, S)` $\rightarrow$ `<L'+R>`  where `L`, `R` are both expressions and `eval_expr(<L>, S)` $\rightarrow$ `L'`;
- `eval_expr(<N + E>, S)` $\rightarrow$ `<N+E'>`  where `N` is a number, `E` is an expression, and `eval_expr(<E>, S)` $\rightarrow$ `E'`;
- `eval_expr(<N + M>, S)` $\rightarrow$ `Q` where `Q = N + M` and `N`, `M` are both numbers;

The rules for all other arithmetic operators (like `-`) follow the same structure as the last three seen here above.

Notice that we are evaluating expressions left to right, and we only perform the underlying sum operation when all sides to be added are constants. This suggests that the `+` operator has two distinct meanings: in one case, it is a placeholder inside the program that encodes the "wish" to perform a sum; in the other case, it is the actual arithmetic operation of adding two numbers together. The context always determines unambiguously which of the two interpretations is applicable: inside angle brackets `<`  and `>`, we manipulate statements and expressions (or shorter just "code"). Outside angle brackets we manipulate state and arithmetic operations (the "concrete computer").

Let us now show some examples of expression evaluation.

`eval_expr(<3>, {})` evaluates a single integer constant. This results in the number three: `3`.

`eval_expr(<3+2>, {})` evaluates the sum of two integer constants. This results in their sum: `5`.

`eval_expr(<10-x>, { x := 5})` must first evaluate the right-hand side, leading us to `eval_expr(<x>, { x := 5})` $\rightarrow$ `5`. This is then injected in place of the value `x`, resulting in a new expression `<10 - 5>` which will be evaluated at the next test.

We can now reformulate the semantics of assignment in order to include the evaluation of expressions when needed. This means that when evaluating an assignment, we will be able to perform a binding in the state right away only when the right-hand side of assignment is a constant number. If this is not the case, we will evaluate the expression before proceeding with the assignment:
- `eval(<V := N>, S)` $\rightarrow$ `<done>, S[V := N]` (`N` is a number);
- `eval(<V := E>, S)` $\rightarrow$ `<V := eval_expr(<E>, S)>, S` (`E` is an expression).

Let us now see an example of expression evaluation and instructions mixed together. We will begin by the very simple program `day := 2; tomorrow := day + 1; done`, evaluated starting from the usual empty state $\{ \}$.

The first step simply adds `day := 2` to the state, since it is the assignment of a constant. This leads us to:

`tomorrow := day + 1; done`, $\{ day := 2 \}$

Evaluating `tomorrow := day + 1` requires more than one step, seen how we must first evaluate the expression `day + 1`, which in turn evaluates the left hand side `day`, which is simply the lookup $\{ day := 2 \}[day]$. This leads us to:

`tomorrow := 2 + 1; done`, $\{ day := 2 \}$

At this point the first statement can be evaluated to a number, since both its operands are constants:

`tomorrow := 3; done`, $\{ day := 2 \}$

Assignment of a single constant leads to a binding, therefore we reach the final step:

`done`, $\{ day := 2, tomorrow := 3 \}$

In the following table, you can find in more details the semantics rules that we follow to evaluate such program:

| Stmt | State | Rule to use |
|:----:|:-----:|:-----------:|
| `<day := 2; tomorrow := day + 1; done>` | $\{ \}$ | `eval(<I; J>, S)` where `eval(<I>, S)` that is `eval(<day := 2>, S)` $\rightarrow$ `<done>, { day := 2 }` |
| `<done; tomorrow := day + 1; done>` | `{ day := 2 }` | `eval(<done; I>, S) → <I>, S` |
| `<tomorrow := day + 1; done>` | `{ day := 2 }` | `eval(<I; J>, S)` where `eval(<I>, S)` is `eval(<tomorrow := day + 1>, S)` |
| `<tomorrow := day + 1; done>` | `{ day := 2 }` | `eval(<V := E>, S)` $\rightarrow$ `<V := eval_expr(<E>, S)>, S` |
| `<tomorrow := eval_expr(<day + 1>, {day := 2}); done>` | `{ day := 2 }` | `eval_expr(<L + R>)` $\rightarrow$ `<L' + R>` where `eval_expr(<L>, S)`$\rightarrow$ `L'` and `eval_expr(<day>, {day := 2})` $\rightarrow$ `{day := 2}[day]` $\rightarrow$ `2` |
| `<tomorrow := 2 + 1; done>` | `{ day := 2 }` | `eval(<V := E>, S)` $\rightarrow$ `<V := eval_expr(<E>, S)>, S` |
| `<tomorrow := eval_expr(<2 + 1>); done>` | `{ day := 2 }` | `eval_expr(<N + M>)` $\rightarrow$ `N+M` |
| `<tomorrow := 3; done>` | `{ day := 2 }` | `eval(<I; J>, S)` where `eval(<I>, S)` that is `eval(<tomorrow := 3>, S)` $\rightarrow$ `<done>, { day := 2; tomorrow := 3 }` |
| `<done; done>` | `{ day := 2; tomorrow := 3 }` | `eval(<done; I>, S) → <I>, S` |
| `<done>` | `{ day := 2; tomorrow := 3 }` | |


## Operator precedence and pattern matching
We can now further extend our language of expressions in order to incorporate new known operators such as multiplication, usually denoted `*` in most programming languages.

Of course this leads to new evaluation rules for the new operator:

- `eval_expr(<L * R>, S)` $\rightarrow$ `<L'*R>`  where `L`, `R` are both expressions and `eval_expr(<L>, S)` $\rightarrow$ `L'`;
- `eval_expr(<N * E>, S)` $\rightarrow$ `<N*E'>`  where `N` is a number, `E` is an expression, and `eval_expr(<E>, S)` $\rightarrow$ `E'`;
- `eval_expr(<N * M>, S)` $\rightarrow$ `Q` where `Q = N` $\times$ `M` and `N`, `M` are both numbers.

This creates ambiguity though: how do we interpret an expression such as `3 + 10 * 2`? On one hand, we could just try our usual strategy of going left-to-right, applying the rule(s) for the evaluation of addition first, and obtaining the evaluation sequence:

| Expr | State |
|:----:|:-----:|
| `<3 + 10 * 2>` | $\{ \}$ |
| `<13 * 2>` | $\{ \}$ |
| `26` | $\{ \}$ |

On the other hand, we know for a fact that multiplication conventionally takes precedence over addition, and as such the evaluation sequence we would want to obtain (in order to be compatible with the basic standards of arithmetics) is:

| Expr | State |
|:----:|:-----:|
| `<3 + 10 * 2>` | $\{ \}$ |
| `<3 + 20>` | $\{ \}$ |
| `23` | $\{ \}$ |

There are multiple ways to enforce the desired behaviour. On one hand, we could adjust the evaluation rules to perform what is known as a _lookup_, that is before evaluating the sum of two terms, we will check to make sure that there are no products that need to be considered further. The crucial evaluation rule to enforce precedence would become: 
- `eval_expr(<A + B * C>, S)` $\rightarrow$ `<A+BC'>`  where `A`, `B`, `C` are all expressions and `eval_expr(<B * C>, S)` $\rightarrow$ `BC'`;

Another solution is based on pre-processing of the input program by a _parser_: a parser is part of the _compiler_ which is an automatic translator that interprets the program. A parser adds brackets around expressions in order to prioritize them (by forcing the evaluator to evaluate the whole block at once: a parenthesized statement matches a whole meta-variable when pattern matching, as discussed in the previous chapter). An expression such as `3 + 10 * 2` would therefore be parenthesized as `3 + (10 * 2)`, and when matching this expression to pattern `L+R` will result in the binding of the meta-variable `L` to 3 and the meta-variable `R` to $10*2$, that is $\{ L := \texttt{3}, R := \texttt{10 * 2} \}$, since parenthesized expressions match in their entirety. 

We will favour the second technique (parsing instead of lookup), for the simple reason that it allows us to keep our evaluation rules easier to write and interpret and we do not have to worry about parentheses because they are added automatically by the parser. 


## Some examples        
Let us now see some examples of evaluating programs featuring complex expressions.

Let us start with program 

```
x := 2 + 3 * 2; done
```

Of course the initial state is empty, that is $\{ \}$.

Evaluation of the program matches the semicolon statement (`<I;J>`), which in turn proceeds with the evaluation of the left-hand statement (in our case `x := 2 + 3 * 2`). The evaluation of the assignment statement evaluates the expression, which had been parenthesized by the parser as `2 + (3 * 2)`. Expression evaluation performs an evaluation step of the right-hand side of the addition, seen that the left-hand side is already a constant. This yields `6` as result, which is then reassembled backwards up until:

```
x := 2 + 6; done
```

Again, with empty state. 

We now perform yet another evaluation step, which matches, again, semicolon, assignment, and sum. The sum of two constants evaluates right away to value `8`, yielding:

```
x := 8; done
```

The evaluation of assignment will now result in a binding in the state, since the right-hand side of the assignment is now a constant. This results in:

```
done
```

with state $\{ x := 8 \}$.

Notice that when evaluating larger programs it will be unwieldy to repeat the whole transformed program after every evaluation step. For this reason, we will often omit the untransformed part of the program by writing an ellipsis (...) instead. This means that the evaluation steps that we have written above would become, in this compact form, as follows:

| Stmt | State |
|:----:|:-----:|
| `<x := 2 + 3 * 2; ...>` | $\{ \}$ |
| `<x := 2 + 6; ...>` | $\{ \}$ |
| `<x := 8; ...>` | $\{ \}$ |
| `<done>` | $\{ x := 8 \}$ |

Let us use this compact form to evaluate yet another program: 

```
bonus := 1; malus := 3; multiplier := 4; score := 2 * multiplier + bonus - malus; done
```
Remember that a parser would add parenthesis to the above program in this way:
```
bonus := 1; malus := 3; multiplier := 4; score := (((2 * multiplier) + bonus) - malus); done
```


The resulting evaluation steps are:

| Stmt | State |
|:----:|:-----:|
| `<bonus := 1; ...>` | $\{ \}$ |
| `<malus := 3; ...>` | $\{ bonus := 1 \}$ |
| `<multiplier := 4; ...>` | $\{ bonus := 1, malus := 3 \}$ |
| `<score := 2 * multiplier + bonus - malus; ...>` | $\{ bonus := 1, malus := 3, multiplier := 4 \}$ |
| `<score := 2 * 4 + bonus - malus; ...>` | $\{ bonus := 1, malus := 3, multiplier := 4 \}$ |
| `<score := 8 + bonus - malus; ...>` | $\{ bonus := 1, malus := 3, multiplier := 4 \}$ |
| `<score := 8 + 1 - malus; ...>` | $\{ bonus := 1, malus := 3, multiplier := 4 \}$ |
| `<score := 9 - malus; ...>` | $\{ bonus := 1, malus := 3, multiplier := 4 \}$ |
| `<score := 9 - 3; ...>` | $\{ bonus := 1, malus := 3, multiplier := 4 \}$ |
| `<score := 6; ...>` | $\{ bonus := 1, malus := 3, multiplier := 4 \}$ |
| `<done>` | $\{ bonus := 1, malus := 3, multiplier := 4, score := 6 \}$ |

The evaluation table also evidences the subtle nature of evaluation. Evaluation identifies the leftmost statment or expression which can be trivially evaluated (a constant, the sum of two constants, the assignment of a constant to a variable) and when this is not possible, then we look in-depth by only taking left turns.


## Expressive power of _Var_ vs _Turtle_
Let us now consider the states we can reach by means of _Turtle_. In _Turtle_, we are limited to only states in the form:

$$\{ x := \dots, y := \dots, pen := \dots \}$$

This gives us a network of possible states looking like:

<img src="images/turtle_state_transitions.png" alt="Turtle state transitions" style="width: 400px;"/>

The network of possible states reachable via _Turtle_ programs is therefore quite limited: no states such as 

$$\{ x := \dots, y := \dots, z := \dots \}$$

can be constructed by running _Turtle_ programs.

On the other hand, the states we can build with _Var_ are, simply said, many more, since we can have states containing as many variables as we want: we just need to assign them! 

This means that all states reachable with _Turtle_ can be reached by _Var_, but not the other way around. The intuition that _Var_ is more expressive than _Turtle_ can be formalized by means of an _embedding_ (semantics-preserving translation, also known as _functor_) from _Turtle_ programs into _Var_ programs. We will call our translation $F$, which transforms programs of _Turtle_ into programs of _Var_:

- $F($ `<I;J>` $) \rightarrow F($ `<I>` $); F($ `<J>` $)$
- $F($ `<up N>` $) \rightarrow $ `<y := y + N>`
- $F($ `<right N>` $) \rightarrow $ `<x := x + N>`
- ...
- $F($ `<done>` $) \rightarrow $ `<done>`

It is easy to see that any program of _Turtle_ can be translated this way into a perfectly equivalent _Var_ program, whereas such a translation cannot exist from _Var_ to _Turtle_. 

At this point, we can also make a minor observation on the nature of our discipline. If this were a Computer Science book, then we would now ignore our intuition about the assertion above that $F$ preserves semantics, and move on to formally demonstrate this via induction. Such a proof is interesting for the theoretician, and adds deep insights into the core of the discipline, but is quite useless for the software engineer for whom a solid intuitive understanding is more than enough to go on building reliable systems. Since this is a software engineering book, we will stop here and move on to the next chapter.